CHARGEMENT DES DONNÉES

CHARGEMENT DES DONNEES

In [ ]:
# IMPORTATIONS DES MODULES
! pip install torch
!pip install gdown
! pip install options
! pip install segmentation-models-pytorch

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

class LEVIRCDClassificationDataset(Dataset):
    def __init__(self, img_dir_before, img_dir_after, mask_dir, transform=None):
        self.img_dir_before = img_dir_before
        self.img_dir_after = img_dir_after
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir_before)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_before_path = os.path.join(self.img_dir_before, img_name)
        img_after_path = os.path.join(self.img_dir_after, img_name)
        mask_path = os.path.join(self.mask_dir, img_name)

        img_before = Image.open(img_before_path).convert("RGB")
        img_after = Image.open(img_after_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Grayscale mask

        # Calculer le label binaire : 1 s'il y a un changement, 0 sinon
        label = 1 if torch.sum(transforms.ToTensor()(mask)) > 0 else 0

        if self.transform:
            img_before = self.transform(img_before)
            img_after = self.transform(img_after)

        # Retourner les images avant, après, et le label binaire
        return torch.cat([img_before, img_after], dim=0), label

# Transforms pour normaliser les images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Chemins vers les répertoires de données
img_dir_before = '/content/LEVIR-CD/train/A'
img_dir_after = '/content/LEVIR-CD/train/B'
mask_dir = '/content/LEVIR-CD/train/label'

# Charger les données
train_dataset = LEVIRCDClassificationDataset(img_dir_before, img_dir_after, mask_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [ ]:
# Fonction de perte et optimiseur
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Boucle d'entraînement
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        labels = labels.squeeze(1)   # Adapter les labels pour BCELoss

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passage en avant
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Passage en arrière et optimisation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Époque {epoch+1}/{num_epochs}, Perte: {running_loss/len(train_loader)}")

print("Entraînement terminé.")


Époque 1/2, Perte: 10.491071428571429
Époque 2/2, Perte: 10.491071428571429
Entraînement terminé.


In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np

# Fonction pour calculer mIoU
def compute_miou(all_labels, all_preds, num_classes=2):
    # Calcul de l'IoU pour chaque classe
    IoUs = []
    for cls in range(num_classes):
        intersection = np.sum((all_preds == cls) & (all_labels == cls))
        union = np.sum((all_preds == cls) | (all_labels == cls))
        IoU = intersection / union if union > 0 else 0
        IoUs.append(IoU)
    mIoU = np.mean(IoUs)
    return mIoU

# Fonction pour évaluer le modèle et sauvegarder les résultats
def evaluate_model(model, data_loader, dataset_name):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            preds = (outputs > 0.5).int()  # Convertir en prédictions binaires
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcul des métriques
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    # Calcul des scores SCS et SC
    binary_change_score = np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    semantic_change_score = np.sum(np.array(all_preds) & np.array(all_labels)) / np.sum(all_labels) if np.sum(all_labels) > 0 else 0
    mIoU = compute_miou(np.array(all_labels), np.array(all_preds))

    # Affichage des résultats
    print(f"{dataset_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, "
          f"BC: {binary_change_score:.4f}, SC: {semantic_change_score:.4f}, mIoU: {mIoU:.4f}")

    # Sauvegarde des résultats dans un fichier
    with open("evaluation_results.txt", "a") as f:
        f.write(f"{dataset_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, "
                 f"BC: {binary_change_score:.4f}, SC: {semantic_change_score:.4f}, mIoU: {mIoU:.4f}\n")

# Charger les données de test
test_dataset = LEVIRCDClassificationDataset('/content/LEVIR-CD/test/A', '/content/LEVIR-CD/test/B', '/content/LEVIR-CD/test/label', transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Charger les données de validation
val_dataset = LEVIRCDClassificationDataset('/content/LEVIR-CD/val/A', '/content/LEVIR-CD/val/B', '/content/LEVIR-CD/val/label', transform)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Évaluer le modèle sur le jeu de test
evaluate_model(model, test_loader, dataset_name="Test")

# Évaluer le modèle sur le jeu de validation
evaluate_model(model, val_loader, dataset_name="Validation")


Test - Accuracy: 0.9219, Precision: 0.9219, Recall: 1.0000, BC: 118.0000, SC: 128.0000, mIoU: 0.4609
Validation - Accuracy: 0.8750, Precision: 0.8750, Recall: 1.0000, BC: 56.0000, SC: 64.0000, mIoU: 0.4375
